In [20]:
import os, nltk, re
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import NLTKTextSplitter,RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain import PromptTemplate
from pathlib import Path as p
os.environ["OPENAI_API_KEY"]=open(r"C:\Users\shaur\Downloads\chat_gpt_key.txt",'r').read()
os.chdir(r"C:\Users\shaur\Desktop\icici policies wording")
os.getcwd()

'C:\\Users\\shaur\\Desktop\\icici policies wording'

In [3]:
all_policies_pdfs = [i for i in os.listdir() if '.pdf' in i]
all_policies_dict = {}
for i in range(0,len(all_policies_pdfs)):
    loader = PyPDFLoader(all_policies_pdfs[i])
    pages = loader.load()
    pages = [page.page_content for page in pages]
    print(f"Length of pages from {all_policies_pdfs[i]}:- {len(pages)}")
    all_policies_dict[all_policies_pdfs[i]] = pages
all_policies_dict.keys()

Length of pages from Elevate Policy Wordings.pdf:- 45
Length of pages from Health AdvantEdge_Policy wordings_IL.pdf:- 51
Length of pages from policy-wordings_maxprotect.pdf:- 40


dict_keys(['Elevate Policy Wordings.pdf', 'Health AdvantEdge_Policy wordings_IL.pdf', 'policy-wordings_maxprotect.pdf'])

In [22]:
all_policies_dict['Elevate Policy Wordings.pdf'][0:6]

['a. PREAMBLE:\n  ICICI Lombard General Insurance Company Limited \n(“We/Us”), having received a Proposal and the premium \nfrom the Proposer named in Part A of the Policy \n(hereinafter referred to as the “Policy Schedule”) and the \nsaid Proposal form with any statement, report or other \ndocument leading to the issue of this Policy and referred \nto therein having been accepted and agreed to by Us \nand the Proposer as the basis of this contract do, by this \nPolicy agree, in consideration of and subject to the due \nreceipt of the subsequent premiums, as set out in the \nPolicy Schedule. \n  Further, subject to the Policy terms and conditions that \non proof to Our satisfaction of the compensation having \nbecome payable as set out in the Policy Schedule to the \nsaid person or persons claiming payment or occurencean \nevent upon which one or more benefits become payable \nunder this Policy, the Annual Sum Insured / appropriate \nbenefit amount will be paid by Us.\nb. DEFINITIONS:\

In [23]:
all_policies_dict['Health AdvantEdge_Policy wordings_IL.pdf'][0:6]

['Policy Wordings  \n \nb. PREAMBLE:  \nICICI Lombard General Insurance Company Limited (“We / Us”), having received a Proposal and the premium from the \nProposer named in Part a of the Policy (hereinafter referred to as the “Policy Schedule”) and the said Proposal and \nDeclaration together with any statement, report or other document leading to the issue of this Policy and referred to \ntherein having been accepted and agreed to by Us and the Prop oser as the basis of this contract do, by this Policy \nagree, in consideration of and subject to the due receipt of the subsequent premiums, as set out in the Policy Schedule, \nand further, subject to the terms and conditions contained in this Policy that on  proof to Our satisfaction of the \ncompensation having become payable as set out in the Policy Schedule to the title of the said person or persons claiming \npayment or upon the happening of an event upon which one or more benefits become payable under this P olicy, the \nsum insure

In [24]:
all_policies_dict['policy-wordings_maxprotect.pdf'][0:6]

['II. Preamble  \nThis Policy is proof of the contract between you and us. The declarations, disclosures and \nconsents given by you in the Proposal is also a part of this contract.  This Policy testifies that \nwe will insure your interests under the Sections specified as operative in the Policy certificate \nbased on the premium paid by you to us. It confirms that we will indemnify you for the events \noccurring during the Period of Insur ance in the manner and to the extent specified in the \nPolicy. For this, it is vital that you meet, do and comply with anything related to the terms, \nconditions and exceptions of this Policy . \nIII. Definitions  \nFor the purposes of this Policy, the terms specified below shall have the meaning set forth \nwherever appearing/specified in this Policy or related Extensions:  \nWhere the context so requires, references to the singular shall also include references to \nthe plural and references to any gender shall include references to all genders.

In [7]:
repeted_text_0 = 'IRDA Reg. No. 115\nMailing Address:\n601 / 602, 6th Floor, Interface Building No. 16,\nNew Link Road, Malad (West), \nMumbai - 400 064.CIN:  L67200MH2000PLC129408\nRegistered Office Address:\nICICI Lombard House, 414, P Balu Marg, Off \nVeer Savarkar Road, Nr Siddhi Vinayak Temple, \nPrabhadevi, Mumbai - 400 025.UIN: ICIHLIP25048V042425 Product Name: Elevate\nToll free No.  : 1800 2666  \nAlternate No.: 86552 22666 (Chargeable)\nWebsite :  www.iciclombard.com\nE-mail : customersupport@icicilombard.comICICI Lombard General Insurance Company Limited'
repeted_text_1 = 'Health AdvantEdge  \nICICI Lombard General Insurance Company Limited  \n       IRDA  Reg. No. 115                            CIN : L67200MH2000PLC129408                                  UIN: ICIHLIP24182V042324     Health A dvantEdge  \n       Mailing A ddress:                              Registered Office A ddress:                                Toll free no : 1800 2666   \n601 & 602, 6th Floor, Interf ace 16,    ICICI Lombard House, 414, P Balu Marg,                     A lternate no :  86552 2 2666 (chargeable)   \nNew Linking Road, Malad (West)      Of f  Veer Sav arkar Road, Nr Siddhi Vinay ak Temple,    E-mail : customersupport@icicilombard.com  \n    Mumbai - 400 064                             Prabhadev i, Mumbai 400 025                                        Website : www.icicilombard.com   '
repeted_text_2 = 'Annexure I  MaxProtect  Policy w ordings  \nIRDAI reg. no.: 115                CIN: L67200MH20000PLC129408   UIN: ICIHLIP24084V012324'
for i in all_policies_dict.keys():
    for k in [repeted_text_0,repeted_text_1,repeted_text_2]:
        all_policies_dict[i] = [page.replace(k.strip(), '').strip() for page in all_policies_dict[i]]

In [8]:
# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1500,
#     chunk_overlap=200,
#     separators=["\n\n", "\n", ".", " "]
# )
splitter = NLTKTextSplitter(chunk_size=1200, chunk_overlap=200)
chunked_policies = {}
for i, full_text in all_policies_dict.items():
    full_text = "\n".join(pages)
    docs = splitter.create_documents([full_text])
    chunked_policies[i] = docs

Created a chunk of size 1338, which is longer than the specified 1200
Created a chunk of size 1338, which is longer than the specified 1200
Created a chunk of size 1338, which is longer than the specified 1200


In [9]:
print(chunked_policies['policy-wordings_maxprotect.pdf'][6].page_content)

A co-payment does not reduce the Sum Insured.

Cumulative Bonus  means any increase or a ddition in the Sum Insured granted by the insurer 
without an associated increase in premium  
 
Day Care Centre  means any institution established for day care treatment of illness and/or 
injuries or a medical setup with a hospital and which has been registere d with the local 
Annexure I  MaxProtect  Policy w ordings  
IRDAI reg.

no.

: 115                CIN: L67200MH20000PLC129408   UIN: ICIHLIP24084V012324  authorities, wherever applicable, and is under supervision of a registered and qualified medical 
practitioner AND must comply with all minimum criterion as under – 
i. has qualified nursing staff under its employment;  
ii.

has qualified medical practitioner/s in charge  
iii.

has fully equipped operation theatre of its own where surgical procedures are carried 
out; 
iv.

maintains daily records of patients and will make these accessible to the insurance 
company’s authorized personne

In [10]:
chunked_policies.keys()

dict_keys(['Elevate Policy Wordings.pdf', 'Health AdvantEdge_Policy wordings_IL.pdf', 'policy-wordings_maxprotect.pdf'])

In [11]:
for i in chunked_policies.keys():
    print(f"{i}:- {len(chunked_policies[i])}")

Elevate Policy Wordings.pdf:- 120
Health AdvantEdge_Policy wordings_IL.pdf:- 120
policy-wordings_maxprotect.pdf:- 120


In [12]:
# Making an instance just to count numbers of tokens and use summarize to make summary of all policies
# Temperature 0 is for the reason that we do not want it to be creative
llm = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")
total=0
for i in chunked_policies.keys():
    for k in range(0,len(chunked_policies[i])):
        total+=llm.get_num_tokens(chunked_policies[i][k].page_content)
print(f"Total Tokens to be used:- {total}")

Total Tokens to be used:- 87690


In [21]:
# the chain type have 3 type 
# map_reduce (each chuck is summarised and Combines all partial summaries into one final summary), 
# refine (keeps on refining when new chunck is added, much detailed), 
# stuff (load all chucks at onces and used as single prompt to summarize)

map_prompt_template = """
You are an expert in insurance policies. Write a comprehensive summary of this chunk of insurance policy text.
Focus on:
1. Coverage details and limits
2. Exclusions and conditions
3. Key definitions
4. Claim procedures
5. Special terms or benefits

Include any specific numbers, dates, or amounts mentioned.

Text to summarize:
{text}

SUMMARY:
"""
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])
combine_prompt_template = """
As an insurance specialist, create a structured summary of the following insurance policy information delimited by triple backquotes.
Organize your response in clear sections with bullet points covering:

1. POLICY OVERVIEW
2. COVERAGE DETAILS
3. KEY EXCLUSIONS
4. CLAIM PROCESS
5. IMPORTANT DEFINITIONS
6. SPECIAL NOTES

```{text}```

STRUCTURED SUMMARY:
"""
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce",map_prompt=map_prompt,combine_prompt=combine_prompt,return_intermediate_steps=True)

final_results = {}
for policy_name, docs in chunked_policies.items():
    print(f"Summarizing policy: {policy_name}")
    try:
        map_reduce_outputs = summary_chain({"input_documents": docs})
        final_summary = map_reduce_outputs["output_text"]
        
        summary_dir = "policy_summaries"
        os.makedirs(summary_dir, exist_ok=True)
        clean_filename = policy_name.replace('.pdf', '').replace(' ', '_')
        output_path = os.path.join(summary_dir, f"{clean_filename}_summary.txt")
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(final_summary)
        
        print(f"Summary for {policy_name} saved to {output_path}")
    except Exception as e:
        print(f"Error processing {policy_name}: {e}")
        
print("All policy summaries completed!")
# summaries = {}
# for i in chunked_policies:
#     print(f"Summarizing policy index: {i}")
#     summary = summary_chain.invoke(chunked_policies[i])
#     summaries[i] = summary


Summarizing policy: Elevate Policy Wordings.pdf


C:\Users\shaur\AppData\Roaming\Python\Python39\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Error processing Elevate Policy Wordings.pdf: unsupported operand type(s) for +=: 'dict' and 'dict'

Summarizing policy: Health AdvantEdge_Policy wordings_IL.pdf


KeyboardInterrupt: 

In [16]:
summaries

{}